# ML-8. Pipelines

In [1]:
#загрузим основные библиотеки
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
 
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score


Попробуем предсказать цену на недвижимость в Калифорнии

## Часть 1. Простейшие пайплайны


Загрузим данные

In [2]:
data = fetch_california_housing()

In [3]:
df = pd.DataFrame(data['data'], columns=data['feature_names'])
df.loc[:,'target'] = data['target']
#df.describe()
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [4]:
def rmse(y_hat, y):
    return mean_squared_error(y_hat, y, squared = False)

In [5]:
X = df.drop('target', axis=1)
Y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)


In [6]:
print(f'Размер обучающей выборки {X_train.shape}')
print(f'Размер тестовой выборки {X_test.shape}')

Размер обучающей выборки (15480, 8)
Размер тестовой выборки (5160, 8)


In [7]:
pipeline = Pipeline([('scaler', StandardScaler()), ('rf', RandomForestRegressor())])
pipeline.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()), ('rf', RandomForestRegressor())])

In [8]:
y_pred = pipeline.predict(X_test)
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

KeyboardInterrupt: 

In [ ]:
pipeline.get_params()

In [ ]:
print(pipeline[1].n_estimators)
print(pipeline['rf'].n_estimators)

In [ ]:
pipeline.set_params(rf__n_estimators=200)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'scaler__with_mean':[True,False],
              'rf__n_estimators':[100, 200, 500]}
grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose = True)


In [13]:
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
Pipeline(steps=[('scaler', StandardScaler()),
                ('rf', RandomForestRegressor(n_estimators=500))])


In [14]:
y_pred = grid_search.best_estimator_.predict(X_test)
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по метрике R2: 0.81
Качество по RSME: 0.5015


# Часть 2. Предобработка в пайплайнах

In [9]:
df_wine = pd.read_csv('Red.zip')

In [10]:
df_wine.head()

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Pomerol 2011,France,Pomerol,Château La Providence,4.2,100,95.00,2011
1,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016


In [11]:
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8666 entries, 0 to 8665
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             8666 non-null   object 
 1   Country          8666 non-null   object 
 2   Region           8666 non-null   object 
 3   Winery           8666 non-null   object 
 4   Rating           8666 non-null   float64
 5   NumberOfRatings  8666 non-null   int64  
 6   Price            8666 non-null   float64
 7   Year             8666 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 541.8+ KB


In [12]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
ct = make_column_transformer(
     (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']),
    (OrdinalEncoder(), ['Region']))
print(ct)


ColumnTransformer(transformers=[('standardscaler', StandardScaler(), ['Price']),
                                ('onehotencoder', OneHotEncoder(), ['Country']),
                                ('ordinalencoder', OrdinalEncoder(),
                                 ['Region'])])


In [13]:
pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor(random_state=42))])

In [14]:
X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']

In [15]:
pipeline.fit(X, y)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country']),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region'])])),
                ('rf', RandomForestRegressor(random_state=42))])

In [16]:
pd.DataFrame(pipeline['ct'].transform(X).toarray(), columns = ['Price']+ pipeline['ct'].transformers_[1][1].get_feature_names_out().tolist())

ValueError: Shape of passed values is (8666, 32), indices imply (8666, 31)

In [17]:
!pip install joblib
import joblib
joblib.dump(pipeline, 'pipeline_wine.pkl')

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3.9 -m pip install --upgrade pip' command.


['pipeline_wine.pkl']

In [11]:
pipeline_loaded = joblib.load('pipeline.pkl')


In [12]:
print(pipeline_loaded)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf', RandomForestRegressor())])


In [18]:
df_wine_test = pd.read_csv('Red_test.zip')

In [23]:
df_wine_test

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
1,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
2,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
3,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016
4,Capatosta 2015,Italy,Toscana,Poggio Argentiera,3.8,101,19.90,2015
...,...,...,...,...,...,...,...,...
275,Tenuta Tignanello 'Solaia' 2000,Italy,Toscana,Antinori,4.6,959,486.42,2000
276,Toscana Il Pareto 2013,Italy,Toscana,Tenuta di Nozzole,4.1,96,48.40,2013
277,Col di Sasso (Cabernet Sauvignon - Sangiovese)...,Italy,Toscana,Banfi,3.5,969,7.25,2018
278,Bramaluce Toscana Rosso 2015,Italy,Toscana,Terenzi,3.8,98,13.90,2015


In [22]:
X_test = df_wine[['Country', 'Price', 'Region']]
y_test = df_wine['Rating']

In [23]:
y_pred = pipeline.predict(X_test)

In [25]:
print(f'Качество по метрике R2: { round(r2_score(y_test, y_pred),4)}')
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по метрике R2: 0.923
Качество по RSME: 0.0856


In [26]:
df_wine = pd.read_csv('Red.zip')
X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']
 
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
     (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']),
    )
 
pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor(random_state=42))])
pipeline.fit(X,y)
 
df_wine_test = pd.read_csv('Red_test.zip')
X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по RSME: 0.0765


In [27]:
pipeline.set_params(rf__n_estimators=200)

Pipeline(steps=[('ct',
                 ColumnTransformer(transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['Region']),
                                                 ('standardscaler',
                                                  StandardScaler(), ['Price']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Country'])])),
                ('rf',
                 RandomForestRegressor(n_estimators=200, random_state=42))])

In [28]:
pipeline.fit(X,y)
 
df_wine_test = pd.read_csv('Red_test.zip')
X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

Качество по RSME: 0.0761


In [36]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeRegressor

In [37]:
#Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('lr', RidgeCV()),
    ('dtr',  DecisionTreeRegressor(random_state=42))
]

#Создаем объект класса стекинг
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)


In [40]:
from sklearn.preprocessing import OrdinalEncoder
df_wine= pd.read_csv('Red.zip')
X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']
 
ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
     (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']),
    )
 
pipeline = Pipeline([('ct', ct), ('stack',reg) ])
pipeline.fit(X,y)
 
 
df_wine_test = pd.read_csv('Red_test.zip')
X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),5)}')

Качество по RSME: 0.18304


In [ ]:
StackingRegressor